# TeAAL specifications on variants of Loops' SpMM implementations (Thread-Mapped)

Description: Assign a fixed, constant number of work tiles to each thread. Resultant work items from each work tile are processed sequentially within the thread. 

In this version, each `tile` represents a row of $A$, and each `atom` represents a nonzero entry of $A$. In other words, assigning a thread to each row of $A$. Therefore, TeAAL specs is idential to the original.

Template: https://github.com/gunrock/loops/blob/main/include/loops/algorithms/spmm/thread_mapped.cuh

Scheduler (referenced as `config` below): https://github.com/gunrock/loops/blob/main/include/loops/schedule/thread_mapped.hxx

GPU Kernel Template (Use it as a reference, don't execute it):

In [ ]:
__global__ void __thread_mapped(...) {
  for (auto A_row : config.tiles()) { # Loop over assigned rows for each tid.
    for (auto B_col : custom_stride_range(std::size_t(0), B.cols, std::size_t(1))) { # Loop over every column of B
      type_t sum = 0;
      for (auto A_nz_idx : config.atoms(A_row)) { # Loop over nonzero entry of current row of A and multiply with corresponding nonzero entry of B.
        sum += A_values[A_nz_idx] * B(indices[A_nz_idx], B_col);
      }

      # Output
      Z(A_row, B_col) = sum;
    }
  }
}

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "../../")

from src import utils

## Initialization

Initialize the input tensors.

For simplicity, the size of a thread warp is the same as the size of a thread block (`WARP_SIZE = BLOCK_SIZE`). Suppose that each GPU SM processes 1 thread warp/block per cycle.

In [ ]:
K = 4
M = 4
N = 4

# Hardware Specification
NUM_SM = 2 # Number of GPU SMs 
WARP_SIZE = 2 # Number of threads per warp
NUM_THREADS = NUM_SM * WARP_SIZE # Total number of threads

print(f"Hardware Specification\n  NUM_SM: {NUM_SM}, WARP_SIZE: {WARP_SIZE}, NUM_THREADS: {NUM_THREADS}")

seed = 1

A_MK = Tensor.fromRandom(rank_ids=["M", "K"], shape=[M, K], seed=seed, density=[0.9, 0.6], name="A") # SpMM, A is sparse
B_KN = Tensor.fromRandom(rank_ids=["K", "N"], shape=[K, N], seed=seed + 1, density=[1, 1], name="B") # SpMM, B is dense

## TeAAL Specifications

Rows of matrix $A$ are partitioned across the SMs' threads. A thread can be assigned to a row with all zeros. 

Note that the current TeAAL specificaiton does not allow to specify the rank of `opt: slip`. This means there exists a synchronization across the SMs.

In [ ]:
yaml = """
einsum:
  declaration:
    A: [M, K]
    B: [K, N]
    Z: [M, N]
  expressions:
    - Z[m, n] = A[m, k] * B[k, n]
mapping:
  rank-order:
    A: [M, K]
    B: [K, N]
    Z: [M, N]
  partitioning:
    Z:
      M: [uniform_shape(NUM_THREADS)]
  loop-order:
    Z: [M1, M0, N, K]
    # M1: Number of partitioned rows of A
    # M0: Size of each partitioned row of A = NUM_THREADS
  spacetime:
    Z:
      space: [M0]
      time: [M1, N, K] 
"""

utils.compile(yaml)

## Check Results

Check that generated code computes the correct result.

**Note**: Should be used after compiling and running the kernel (above cell).

In [ ]:
utils.check_matmul(A_MK, B_KN, Z_MN)

## Performance on GPU

Load Balance: Poor load balance due to the difference in NNZ per row of $A$. This results in threads that are assigned to rows with few NNZ being idle.

Assuming that the data is stored in a row-wise order, the memory access pattern would be:
- A: Uncoalesced access, threads in a warp are accessing different rows of $A$.
- B: Uncoalesced access, threads are accessing $B$ in column-wise order and at random positions based on column indices of $A$'s nonzero entries.
- Z: Uncoalesced access, threads in a warp are writing to different rows of $Z$.